In [1]:
# check syft installation
!pip install syft --pre

In [1]:
import syft as sy
import numpy as np
import pandas as pd

/opt/anaconda3/envs/syft_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/CarlaFC/Star-Wars/main/characters.csv')
df.columns

Index(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color',
       'birth_year', 'gender', 'homeworld', 'species'],
      dtype='object')

In [27]:
# make mass a numeric column
df['mass'] = pd.to_numeric(df['mass'], errors='coerce')

# add column for characters from Naboo
df['from_naboo'] = pd.to_numeric(df.homeworld=='Naboo')
df.sample(5)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,from_naboo
50,Eeth Koth,171.0,NaN,black,brown,brown,NaN,male,Iridonia,Zabrak,False
35,Rugor Nass,206.0,NaN,none,green,orange,NaN,male,Naboo,Gungan,True
81,Finn,NaN,NaN,black,dark,dark,NaN,male,NaN,Human,False
46,Ben Quadinaros,163.0,65.0,none,"grey, green, yellow",orange,NaN,male,Tund,Toong,False
76,Grievous,216.0,159.0,none,"brown, white","green, yellow",NaN,male,Kalee,Kaleesh,False


In [28]:
data_subjects = sy.DataSubjectArray.from_objs(df["name"])
data_subjects[:5]

array([DataSubjectArray: {'Luke Skywalker'}, DataSubjectArray: {'C-3PO'},
       DataSubjectArray: {'R2-D2'}, DataSubjectArray: {'Darth Vader'},
       DataSubjectArray: {'Leia Organa'}], dtype=object)

In [29]:
mass_data = sy.Tensor(df["mass"]).annotated_with_dp_metadata(
   min_val=df.mass.min(), max_val=df.mass.max(), data_subjects=data_subjects
)

Tensor annotated with DP Metadata


In [30]:
from_naboo = sy.Tensor(df.homeworld=='Naboo').annotated_with_dp_metadata(
   min_val=0, max_val=1, data_subjects=data_subjects
)

Tensor annotated with DP Metadata


In [31]:
domain_client=sy.login(url="20.245.180.89",
                       email="kenobi@jedi.org",
                      password="hellothere", )

Connecting to 20.245.180.89... done! 	 Logging into openmined_hospital... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.47
Version on the node: 0.7.0-beta.46



In [32]:
domain_client.load_dataset(
   name="Star_Wars_Characters",
   assets={
      "mass": mass_data,
       "from_naboo": from_naboo
   },
   description="Kenobi's census"
)

Loading dataset... uploading...🚀                        

Uploading `from_naboo`: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00,  7.59it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [33]:
domain_client.datasets

Idx,Name,Description,Assets,Id
[0],OMTissueMNISTDataset,Tissue Specimen Dataset upload from OM uploaded by Osam :).,"[""images""] -> [""labels""] ->",d40b55bb-f0a0-4db3-81c2-144c3d10ab2f
[1],OMTissueMNISTDataset-jb,Tissue Specimen Dataset upload from OM.,"[""images""] -> [""labels""] ->",a0814ae7-4cdf-4ac4-9fb0-097752199f9d
[2],OMTissueMNISTDataset-af,Tissue Specimen Dataset upload from OM.,"[""images""] -> [""labels""] ->",a77fe16c-7f90-453d-bc40-f945b596fe4b
[3],OMTissueMNISTDataset,Tissue Specimen Dataset upload from OM.,"[""images""] -> [""labels""] ->",6ea9f58c-4f84-4524-9222-7c0e03aac05f
[4],Star_Wars_Characters,Test dataset of characters from star wars and their mass,"[""mass""] ->",ae2ba09c-56f3-4682-bf3e-74171f34ba30
[5],Star_Wars_Characters,Kenobi's census,"[""mass""] -> [""from_naboo""] ->",e530d303-50f9-40d6-8032-b1ec4cd31dc5
